## Goal - Extract Lines Vertex and put them in a new shapefile

In [ ]:
import os
from osgeo import ogr

In [ ]:
inshp = '/mnt/x/pwgis/code24data/demo2gdal/roads.shp'
oushp = '/mnt/x/pwgis/code24data/results/demo2gdal.shp'

In [ ]:
"""
GDAL Drivers Name
"""

def drv_name(_file):
    """
    Return the driver for a given file format
    """
    
    drv = {
        # Vector files
        '.gml'    : 'GML',
        '.shp'    : 'ESRI Shapefile',
        '.json'   : 'GeoJSON',
        '.kml'    : 'KML',
        '.osm'    : 'OSM',
        '.dbf'    : 'ESRI Shapefile',
        '.vct'    : 'Idrisi',
        '.nc'     : 'netCDF',
        '.vrt'    : 'VRT',
        '.mem'    : 'MEMORY',
        '.sqlite' : 'SQLite',
        '.gdb'    : 'FileGDB',
        # Raster files
        '.tif'    : 'GTiff',
        '.ecw'    : 'ECW',
        '.mpr'    : 'ILWIS',
        '.mpl'    : 'ILWIS',
        '.jpg'    : 'JPEG',
        '.nc'     : 'netCDF',
        '.png'    : 'PNG',
        '.vrt'    : 'VRT',
        '.asc'    : 'AAIGrid',
        # Vector or Raster
        '.gpkg'   : 'GPKG'
    }
    
    return str(drv[os.path.splitext(_file)[1]])

In [ ]:
def get_shp_sref(shp):
    """
    Get Spatial Reference Object from Feature Class/Lyr
    """
    
    if type(shp) == ogr.Layer:
        lyr = shp
        
        c = 0
    
    else:
        data = ogr.GetDriverByName(
            drv_name(shp)).Open(shp)
        
        lyr = data.GetLayer()
        c = 1
    
    spref = lyr.GetSpatialRef()
    
    if c:
        del lyr
        data.Destroy()
    
    return spref

In [ ]:
# Open Input
polyData = ogr.GetDriverByName(drv_name(inshp)).Open(inshp)
    
polyLyr = polyData.GetLayer()
    
# Get SRS for the output
srs = get_shp_sref(polyLyr)
    
# Create Output
pntData = ogr.GetDriverByName(
    drv_name(oushp)).CreateDataSource(oushp)
    
pntLyr = pntData.CreateLayer(
    os.path.splitext(os.path.basename(oushp))[0],
    srs, geom_type=ogr.wkbPoint
)
    
# Copy fields from input to output
inDefn = polyLyr.GetLayerDefn()

field_names = []
for i in range(0, inDefn.GetFieldCount()):
    fDefn = inDefn.GetFieldDefn(i)
    pntLyr.CreateField(fDefn)
    field_names.append(fDefn.name)

# Create ofid and cpnt
pntLyr.CreateField(ogr.FieldDefn("ofid", ogr.OFTInteger))
pntLyr.CreateField(ogr.FieldDefn("cpnt", ogr.OFTInteger))
    
# Polyline Vertex to Point Feature Class
pntLyrDefn = pntLyr.GetLayerDefn()
for feat in polyLyr:
    geom = feat.GetGeometryRef()
        
    # Get point count
    nrPnt = geom.GetPointCount()
        
    # Add point to a new feature
    for p in range(nrPnt):
        x, y, z = geom.GetPoint(p)
            
        new_point = ogr.Geometry(ogr.wkbPoint)
        new_point.AddPoint(x, y)
            
        new_feature = ogr.Feature(pntLyrDefn)
        new_feature.SetGeometry(new_point)
        
        new_feature.SetField("ofid", feat.GetFID())
        new_feature.SetField("cpnt", p)
            
        for f in field_names:
            new_feature.SetField(f, feat.GetField(f))
            
        pntLyr.CreateFeature(new_feature)
            
        new_feature.Destroy()
    
del pntLyr
del polyLyr
pntData.Destroy()
polyData.Destroy()